In [1]:
#初始化
import numpy as np
import os
import tarfile
import sklearn
import csv
import math
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
plt.rcParams["font.family"]="Times New Roman"
np.random.seed(42)
tf.random.set_seed(42)
from scipy.interpolate import splev
import seaborn as sns
import os

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [40]:
#读取数据
database_name="data2"
# database_name="all"

model_name="BPNN"
def load_data(name):
    location="./data/"+name+".csv"
    data=pd.read_csv(location,sep=",").iloc[:,:]
    return data

In [41]:
data_org=load_data(database_name)
data_x,data_y=data_org.iloc[:,:-1],data_org.iloc[:,-1]

In [92]:
ratio=0.25

In [93]:
#划分测试集（暂定随机抽样）
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
x_train_full, x_test, y_train_full, y_test = train_test_split(data_x,data_y,test_size=ratio, random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full, test_size=0.15,random_state=42)

In [94]:
print(x_train.shape,x_test.shape,x_valid.shape)

(99, 3) (39, 3) (18, 3)


In [95]:
#特征缩放
scaler = StandardScaler()
x_train_t = scaler.fit_transform(x_train)
x_valid_t = scaler.transform(x_valid)
x_test_t = scaler.transform(x_test)

In [96]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(10, activation="softplus", input_shape=x_train_t.shape[1:]))
model.add(keras.layers.Dense(20, activation="softplus"))
model.add(keras.layers.Dense(1))

#通过earlystop，发现模型并未进一步变好则停止，使用checkpoint保存模型
# checkpointCallback=keras.callbacks.ModelCheckpoint("./model/keras_model.h5",save_best_only=True)
# earlystopCallback=keras.callbacks.EarlyStopping(patience=100,restore_best_weights=True)
#但尝试后earlystop发现并不适合该问题，该问题数据量太少，导致验证集表现起伏较大
#通过该类，实现当val_loss足够小时停止
class haltCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if((logs.get('loss') <=7e-04) and (logs.get("val_loss")<=1e-04)):
#         if(logs.get('val_loss') <= 0.0005):

            print("\n\n\ncancelling training!\n\n\n")
            self.model.stop_training = True
trainingStopCallback = haltCallback()

sgd=keras.optimizers.SGD(lr=0.04, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="mean_squared_error", optimizer=sgd)
history = model.fit(x_train_t, y_train, epochs=10000, validation_data=(x_valid_t, y_valid),
                    callbacks=[trainingStopCallback])
                    
#                     callbacks=[trainingStopCallback,checkpointCallback,earlystopCallback])
# model=keras.models.load_model("./model/keras_model.h5")
mse_test = model.evaluate(x_test_t, y_test)
print(mse_test)

A:\1-Applications\1-anaconda\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10000
4/4 [==============================] - 0s 51ms/step - loss: 0.3269 - val_loss: 0.1073
Epoch 2/10000
4/4 [==============================] - 0s 20ms/step - loss: 0.1725 - val_loss: 0.0817
Epoch 3/10000
4/4 [==============================] - 0s 16ms/step - loss: 0.0647 - val_loss: 0.0134
Epoch 4/10000
4/4 [==============================] - 0s 15ms/step - loss: 0.0150 - val_loss: 0.0139
Epoch 5/10000
4/4 [==============================] - 0s 20ms/step - loss: 0.0178 - val_loss: 0.0110
Epoch 6/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.0208 - val_loss: 0.0245
Epoch 7/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.0239 - val_loss: 0.0052
Epoch 8/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.0107 - val_loss: 0.0538
Epoch 9/10000
4/4 [==============================] - 0s 9ms/step - loss: 0.0292 - val_loss: 0.0095
Epoch 10/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.0145 - val_loss: 0.0077
Epoc

Epoch 83/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.0091 - val_loss: 0.0069
Epoch 84/10000
4/4 [==============================] - 0s 9ms/step - loss: 0.0066 - val_loss: 0.0030
Epoch 85/10000
4/4 [==============================] - 0s 9ms/step - loss: 0.0058 - val_loss: 0.0102
Epoch 86/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.0111 - val_loss: 0.0021
Epoch 87/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.0083 - val_loss: 0.0034
Epoch 88/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.0095 - val_loss: 0.0018
Epoch 89/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.0061 - val_loss: 0.0028
Epoch 90/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.0062 - val_loss: 0.0049
Epoch 91/10000
4/4 [==============================] - 0s 9ms/step - loss: 0.0068 - val_loss: 0.0073
Epoch 92/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.0085 - val_loss: 0.0126


4/4 [==============================] - 0s 10ms/step - loss: 0.0051 - val_loss: 0.0029
Epoch 165/10000
4/4 [==============================] - 0s 9ms/step - loss: 0.0066 - val_loss: 0.0018
Epoch 166/10000
4/4 [==============================] - 0s 9ms/step - loss: 0.0060 - val_loss: 0.0013
Epoch 167/10000
4/4 [==============================] - 0s 9ms/step - loss: 0.0057 - val_loss: 0.0261
Epoch 168/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.0131 - val_loss: 0.0045
Epoch 169/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.0067 - val_loss: 0.0079
Epoch 170/10000
4/4 [==============================] - 0s 9ms/step - loss: 0.0080 - val_loss: 0.0126
Epoch 171/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.0100 - val_loss: 0.0012
Epoch 172/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.0082 - val_loss: 0.0028
Epoch 173/10000
4/4 [==============================] - 0s 9ms/step - loss: 0.0082 - val_loss: 0.0028
Epoch

4/4 [==============================] - 0s 9ms/step - loss: 0.0068 - val_loss: 0.0263
Epoch 246/10000
4/4 [==============================] - 0s 9ms/step - loss: 0.0146 - val_loss: 0.0013
Epoch 247/10000
4/4 [==============================] - 0s 10ms/step - loss: 0.0049 - val_loss: 0.0017
Epoch 248/10000
4/4 [==============================] - 0s 10ms/step - loss: 0.0046 - val_loss: 0.0016
Epoch 249/10000
4/4 [==============================] - 0s 10ms/step - loss: 0.0045 - val_loss: 0.0038
Epoch 250/10000
4/4 [==============================] - 0s 9ms/step - loss: 0.0050 - val_loss: 0.0021
Epoch 251/10000
4/4 [==============================] - 0s 9ms/step - loss: 0.0049 - val_loss: 0.0023
Epoch 252/10000
4/4 [==============================] - 0s 10ms/step - loss: 0.0046 - val_loss: 0.0085
Epoch 253/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.0073 - val_loss: 0.0014
Epoch 254/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.0049 - val_loss: 0.0027
Ep

...

4/4 [==============================] - 0s 8ms/step - loss: 2.8029e-04 - val_loss: 2.1393e-04
Epoch 3651/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.8913e-04 - val_loss: 1.5527e-04
Epoch 3652/10000
4/4 [==============================] - 0s 10ms/step - loss: 2.7406e-04 - val_loss: 9.2689e-04
Epoch 3653/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.4328e-04 - val_loss: 2.4444e-04
Epoch 3654/10000
4/4 [==============================] - 0s 9ms/step - loss: 3.8376e-04 - val_loss: 1.5639e-04
Epoch 3655/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.1295e-04 - val_loss: 1.5254e-04
Epoch 3656/10000
4/4 [==============================] - 0s 14ms/step - loss: 2.9235e-04 - val_loss: 1.2465e-04
Epoch 3657/10000
4/4 [==============================] - 0s 11ms/step - loss: 2.8410e-04 - val_loss: 1.4313e-04
Epoch 3658/10000
4/4 [==============================] - 0s 9ms/step - loss: 2.9253e-04 - val_loss: 5.4636e-04
Epoch 3659/10000
4/4 

4/4 [==============================] - 0s 8ms/step - loss: 3.3548e-04 - val_loss: 1.6742e-04
Epoch 3725/10000
4/4 [==============================] - 0s 9ms/step - loss: 2.9923e-04 - val_loss: 1.4554e-04
Epoch 3726/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.7518e-04 - val_loss: 1.3414e-04
Epoch 3727/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.7250e-04 - val_loss: 1.6373e-04
Epoch 3728/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.8893e-04 - val_loss: 1.4116e-04
Epoch 3729/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.6609e-04 - val_loss: 3.3719e-04
Epoch 3730/10000
4/4 [==============================] - 0s 8ms/step - loss: 3.1523e-04 - val_loss: 8.4089e-04
Epoch 3731/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.8120e-04 - val_loss: 2.8561e-04
Epoch 3732/10000
4/4 [==============================] - 0s 8ms/step - loss: 3.3618e-04 - val_loss: 1.2286e-04
Epoch 3733/10000
4/4 [=====

4/4 [==============================] - 0s 10ms/step - loss: 2.8568e-04 - val_loss: 1.3080e-04
Epoch 3799/10000
4/4 [==============================] - 0s 11ms/step - loss: 2.6661e-04 - val_loss: 1.2586e-04
Epoch 3800/10000
4/4 [==============================] - 0s 11ms/step - loss: 2.6232e-04 - val_loss: 1.5379e-04
Epoch 3801/10000
4/4 [==============================] - 0s 11ms/step - loss: 2.7285e-04 - val_loss: 0.0018
Epoch 3802/10000
4/4 [==============================] - 0s 9ms/step - loss: 0.0011 - val_loss: 0.0022
Epoch 3803/10000
4/4 [==============================] - 0s 10ms/step - loss: 0.0013 - val_loss: 3.2705e-04
Epoch 3804/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.9655e-04 - val_loss: 1.3337e-04
Epoch 3805/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.7042e-04 - val_loss: 1.3589e-04
Epoch 3806/10000
4/4 [==============================] - 0s 10ms/step - loss: 3.4732e-04 - val_loss: 1.7140e-04
Epoch 3807/10000
4/4 [=============

4/4 [==============================] - 0s 9ms/step - loss: 3.1220e-04 - val_loss: 1.2356e-04
Epoch 3873/10000
4/4 [==============================] - 0s 9ms/step - loss: 2.4971e-04 - val_loss: 1.1267e-04
Epoch 3874/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.4247e-04 - val_loss: 1.2660e-04
Epoch 3875/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.5745e-04 - val_loss: 2.0140e-04
Epoch 3876/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.6793e-04 - val_loss: 1.3415e-04
Epoch 3877/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.6184e-04 - val_loss: 2.1821e-04
Epoch 3878/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.6636e-04 - val_loss: 1.2420e-04
Epoch 3879/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.6247e-04 - val_loss: 1.5356e-04
Epoch 3880/10000
4/4 [==============================] - 0s 9ms/step - loss: 2.6028e-04 - val_loss: 1.7730e-04
Epoch 3881/10000
4/4 [=====

4/4 [==============================] - 0s 9ms/step - loss: 3.1901e-04 - val_loss: 1.5546e-04
Epoch 3947/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.8341e-04 - val_loss: 1.3188e-04
Epoch 3948/10000
4/4 [==============================] - 0s 9ms/step - loss: 2.4604e-04 - val_loss: 2.5781e-04
Epoch 3949/10000
4/4 [==============================] - 0s 8ms/step - loss: 3.2339e-04 - val_loss: 2.7653e-04
Epoch 3950/10000
4/4 [==============================] - 0s 7ms/step - loss: 3.0398e-04 - val_loss: 1.2765e-04
Epoch 3951/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.7425e-04 - val_loss: 1.2630e-04
Epoch 3952/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.7158e-04 - val_loss: 1.7978e-04
Epoch 3953/10000
4/4 [==============================] - 0s 7ms/step - loss: 2.7320e-04 - val_loss: 0.0012
Epoch 3954/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.8249e-04 - val_loss: 2.7583e-04
Epoch 3955/10000
4/4 [=========

In [97]:
y_pred = model.predict(x_test_t)
y_train_pred = model.predict(x_train_t)

In [98]:
def relative_error(y_true,y_pred):
    term1=np.sqrt(np.square(y_true-y_pred).sum())
    term2=np.sqrt(np.square(y_true).sum())
    return term1/term2

In [99]:
test_r1=relative_error(y_test, y_pred[:,0])
test_r1

0.020229686994611848

In [100]:
from sklearn.metrics import mean_squared_error#导入计算误差的函数
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
test_r2= r2_score(y_test, y_pred)
test_mse = mean_squared_error(y_test, y_pred)
test_rmse = np.sqrt(test_mse)
print("RMSE",test_rmse)
print("R2",test_r2)

RMSE 0.012735767331718882
R2 0.9971308619431876


In [101]:
y_pred=y_pred.reshape(y_pred.shape[0])
y_train_pred=y_train_pred.reshape(y_train_pred.shape[0])

In [102]:
train_r2= r2_score(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_r1=relative_error(y_train, y_train_pred)
train_r1

0.0245552207548363

In [103]:
def save_test_data():
    data=pd.DataFrame({
        "Ltx":x_test["Ltx"],
        "D":x_test["D"],
        "MI":x_test["MI"],
        "y":y_test.values,
        "pred":y_pred,
        "r1":test_r1,
        "R2":test_r2,
        "RMSE":test_rmse
        
    })
    location="./result/test/performance/"
    name=model_name+"_"+database_name+"_"+str(ratio)+"_test_performance"+".csv"
    data.to_csv(location+name,header=True,mode='w')
    print("done save to",name)
    return data
test_result=save_test_data()

done save to BPNN_data2_0.25_test_performance.csv


In [ ]:
stop

In [104]:
def save_train_data():
    data=pd.DataFrame({
        "Ltx":x_train["Ltx"],
        "D":x_train["D"],
        "MI":x_train["MI"],
        "y":y_train.values,
        "pred":y_train_pred,
        "r1":train_r1,
        "R2":train_r2,
        "RMSE":train_rmse
        
    })
    location="./result/test/performance/"
    name=model_name+"_"+database_name+"_"+str(ratio)+"_train_performance"+".csv"
    data.to_csv(location+name,header=True,mode='w')
    print("done save to",name)
    return data
train_result=save_train_data()

done save to BPNN_data2_0.25_train_performance.csv


In [105]:
aa={"ratio":ratio
                        ,"train":x_train.shape[0]
                        ,"test":x_test.shape[0]
                        ,"valid":x_valid.shape[0]
                        ,"r1":test_r1
                        ,"r2":test_r2
                        ,"RMSE":test_rmse}
aa

{'ratio': 0.25,
 'train': 99,
 'test': 39,
 'valid': 18,
 'r1': 0.020229686994611848,
 'r2': 0.9971308619431876,
 'RMSE': 0.012735767331718882}

In [106]:
data_save=pd.DataFrame({"ratio":ratio
                        ,"train":x_train.shape[0]
                        ,"test":x_test.shape[0]
                        ,"valid":x_valid.shape[0]
                        ,"r1":test_r1
                        ,"r2":test_r2
                        ,"RMSE":test_rmse
                       , "model":model_name},index=[0])
save_loc="./result/result.csv"
data_save.to_csv(save_loc,mode="a+",index=False,header=False)
# data_save.to_csv(save_loc,mode="a+",index=False)

In [ ]:
path='Weights_folder/Weights_'+model_name+"_"+str(round(r1,4))
saveResult = model.save(path)

In [ ]:
x_draw=np.arange(y_pred.shape[0])
fig,ax=plt.subplots(1,1,figsize=(6,4.5))
ax.tick_params(labelsize=12)
ax.plot(x_draw,y_pred,"r-",label="Pred")
ax.plot(x_draw,y_test,"b.",linewidth=5,label="True")
ax.set_xlabel("No",fontsize=15)
ax.set_ylabel("MI",fontsize=15)
ax.set_title(model_name)
ax.legend()

ax.text(0.05,0.95,s=("$R_1$="+str(round(r1,4))),fontsize=12,transform=ax.transAxes)
ax.text(0.05,0.9,s=("$R_2$="+str(round(test_r2,4))),fontsize=12,transform=ax.transAxes)


fig.savefig("./figure/"+model_name+"_"+database_name+"_"+str(round(r1,4))+"_demo.png",dpi=150,
           bbox_inches="tight")

In [ ]:
plt.plot(pd.DataFrame(history.history))
plt.grid(True)
# plt.gca().set_ylim(0, 1)
plt.savefig("./figure/"+model_name+"_"+str(round(r1,4))+database_name+"ln.png",dpi=150)
# plt.savefig("./ln.png")